In [17]:
import pandas as pd
import numpy as np
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
test_scores = []
for i in range(1,6):
    X_train = pd.DataFrame()
    X_test = pd.DataFrame()
    for gp in tqdm([[4,3,2],[8, 4, 2], [13, 1, 3], [12, 8, 2]]):
        new_train = pd.read_csv(f'results/split_spatialS_{i}_{gp}/X_train.csv')
        new_test = pd.read_csv(f'results/split_spatialS_{i}_{gp}/X_test.csv')
        X_train = pd.concat([X_train, new_train], axis=1)
        X_test = pd.concat([X_test, new_test], axis=1)
    y_train = pd.read_csv(f'results/split_spatialS_{i}_{gp}/y_train.csv')
    y_test = pd.read_csv(f'results/split_spatialS_{i}_{gp}/y_test.csv')
    alphas = np.logspace(-6, 6, 20)
    print('start')
    # Define the model and pipeline
    ridge_pipeline = Pipeline([
        # ('scaler', StandardScaler()),
        ('ridge', RidgeCV(alphas=alphas, cv=5, scoring='neg_mean_absolute_error'))
    ])


    ridge_pipeline.fit(X_train, y_train)
    test_score= np.mean(np.abs(ridge_pipeline.predict(X_test)- y_test))
    test_scores.append(test_score)
    print("Test score (MAE):", test_score)

100%|██████████| 4/4 [00:02<00:00,  1.54it/s]


start
Test score (MAE): 0.1554161289806102


100%|██████████| 4/4 [00:04<00:00,  1.06s/it]


start
Test score (MAE): 0.16068112561225867


100%|██████████| 4/4 [00:04<00:00,  1.14s/it]


start
Test score (MAE): 0.16200557977329602


100%|██████████| 4/4 [00:04<00:00,  1.15s/it]


start
Test score (MAE): 0.16801169055165058


100%|██████████| 4/4 [00:04<00:00,  1.13s/it]


start
Test score (MAE): 0.15644389039711698


In [18]:
np.mean(test_scores), np.std(test_scores)/np.sqrt(5)

(0.16051168306298647, 0.002010195603094569)

In [15]:
# Add this at the top of your evaluation script
from scipy.special import sph_harm
import numpy as np

class SphericalHarmonicsEncoder:
    def __init__(self, L=20, output_dim=128):
        self.L = L
        self.output_dim = output_dim
    
    def encode_coordinates(self, lat, lon):
        """
        Encode lat/lon using spherical harmonics
        """
        # Handle single values or arrays
        lat = np.atleast_1d(lat)
        lon = np.atleast_1d(lon)
        
        lat_rad = np.radians(lat)
        lon_rad = np.radians(lon)
        
        harmonics_features = []
        
        for i in range(len(lat)):
            features_i = []
            for l in range(min(self.L + 1, 25)):  # Limit to prevent overflow
                for m in range(-l, l + 1):
                    try:
                        Y_lm = sph_harm(m, l, lon_rad[i], lat_rad[i])
                        features_i.extend([Y_lm.real, Y_lm.imag])
                    except:
                        features_i.extend([0.0, 0.0])  # Fallback for numerical issues
            harmonics_features.append(features_i)
        
        harmonics_array = np.array(harmonics_features)
        
        # Pad or truncate to desired dimension
        if harmonics_array.shape[1] > self.output_dim:
            harmonics_array = harmonics_array[:, :self.output_dim]
        elif harmonics_array.shape[1] < self.output_dim:
            padding = np.zeros((harmonics_array.shape[0], self.output_dim - harmonics_array.shape[1]))
            harmonics_array = np.concatenate([harmonics_array, padding], axis=1)
        
        return harmonics_array

def prepare_location_features(df, use_spherical_harmonics=False):
    """
    Prepare location features from DHS data (already processed)
    """
    all_features = []
    feature_names = []
    
    # hv025 is already one-hot encoded as hv025_1 and hv025_2
    location_columns = ['hv025_1', 'hv025_2']
    
    # Check if columns exist
    missing_cols = [col for col in location_columns if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing location columns: {missing_cols}")
    
    # Extract urban/rural features
    urban_rural_features = df[location_columns].values
    all_features.append(urban_rural_features)
    feature_names.extend(['urban_rural_0', 'urban_rural_1'])
    
    print(f"Urban/Rural feature distribution:")
    print(f"  hv025_1 (Urban): {df['hv025_1'].sum()} samples ({df['hv025_1'].mean()*100:.1f}%)")
    print(f"  hv025_2 (Rural): {df['hv025_2'].sum()} samples ({df['hv025_2'].mean()*100:.1f}%)")

    if use_spherical_harmonics and 'LATNUM' in df.columns and 'LONGNUM' in df.columns:
        print('Encoding coordinates with spherical harmonics...')
        coord_encoder = SphericalHarmonicsEncoder(L=20, output_dim=256)
        
        # Extract coordinates
        lat = df['LATNUM'].values
        lon = df['LONGNUM'].values
        
        # Encode with spherical harmonics
        coord_features = coord_encoder.encode_coordinates(lat, lon)
        all_features.append(coord_features)
        feature_names.extend([f'coord_sh_{i}' for i in range(coord_features.shape[1])])
        
        print(f"  Added spherical harmonics coordinates: {coord_features.shape[1]} features")

        '''
        geographic_features = np.column_stack([
            np.abs(lat),  # Distance to equator
            np.abs(lon),  # Distance to prime meridian
            (np.abs(lat) < 23.5).astype(int),  # Tropical zone indicator
        ])
        all_features.append(geographic_features)
        feature_names.extend(['dist_equator', 'dist_prime_meridian', 'tropical_zone'])
        
        print(f"  Added basic geographic features: {geographic_features.shape[1]} features")
        '''
        
    
    # Optional: Add additional location features if you want to experiment
    additional_features = []
    if additional_features:
        all_features.extend(additional_features)
    
    # Combine all features
    combined_features = np.concatenate(all_features, axis=1)
    
    print(f"  Total location/temporal features: {combined_features.shape[1]}")
    # print(f"  Feature breakdown: Urban/Rural(2) + Coordinates({coord_features.shape[1] if use_spherical_harmonics and 'LATNUM' in df.columns else 0}) + Geographic(3) + Temporal({temporal_features.shape[1] if use_temporal_features and 'year' in df.columns else 0}) + Additional({sum(f.shape[1] for f in additional_features)})")
    
    return combined_features, feature_names


In [16]:
def evaluate(
    fold,
    model_name,
    target="",
    use_checkpoint=False,
    model_not_named_target=True,
    imagery_path=None,
    imagery_source=None,
    mode="temporal",
    model_output_dim=768,
    grouped_bands=None,
    country=None,
    use_location_features=False, # Use Geo info 
    use_spherical_harmonics=False,
):
    model_par_dir = "../model/"
    country_suffix = f'_{country.upper()}' if country else ''

    # Build checkpoint filename (pth file) based on mode and target
    if use_checkpoint:
        named_target = target if model_not_named_target else ""
        if mode == "temporal":
            checkpoint = f"{model_par_dir}{model_name}_temporal_best_{imagery_source}{named_target}{country_suffix}.pth"
        elif mode == "spatial":
            checkpoint = f"{model_par_dir}{model_name}_{fold}_{grouped_bands}all_cluster_best_{imagery_source}{named_target}{country_suffix}.pth"
        elif mode == "one_country":
            checkpoint = f"{model_par_dir}{model_name}_{fold}_one_country_best_{imagery_source}{named_target}{country_suffix}.pth"
        else:
            raise Exception(mode)

    print(
        f"Evaluating {model_name} on fold {fold} {f'for country {country_suffix[1:]}' if country else '' } with target {target} using checkpoint {checkpoint if use_checkpoint else 'None'}"
    )

    # Modified to adjust the actual number of features/column (target size) of the country-wise model
    if use_checkpoint and os.path.exists(checkpoint):
        # Load checkpoint to get the actual target size
        temp_state = torch.load(checkpoint, map_location='cpu')
        actual_target_size = temp_state['model_state_dict']['regression_head.weight'].shape[0]
        target_size = actual_target_size
        print(f"Detected target size from checkpoint: {target_size}")
        
        # Also set eval_target appropriately
        if target == "":
            eval_target = "deprived_sev"  # Use this for single target evaluation
    else:
        # Original logic for when not using checkpoint
        if target == "":
            eval_target = "deprived_sev"
            target_size = 99
        else:
            eval_target = target
            target_size = 1 if model_not_named_target else 99

    # Determine size of target
    '''
    if target == "":
        eval_target = "deprived_sev"
        target_size = 99
    else:
        eval_target = target
        target_size = 1 if model_not_named_target else 99
    '''

    # Image Modify based on the Satellite used (L and S)
    normalization = 30000.0 if imagery_source == "L" else 3000.0
    transform_dim = 336 if imagery_source == "L" else 994

    # DHS data folder
    data_folder = "../../survey_processing/processed_data/"

    # Termporal and Spatial have a different train test spilt
    if mode == "temporal":
        train_df = pd.read_csv(f"{data_folder}before_2020.csv")
        test_df = pd.read_csv(f"{data_folder}after_2020.csv")
    else:
        train_df = pd.read_csv(f"{data_folder}train_fold_{fold}{country_suffix}.csv")
        test_df = pd.read_csv(f"{data_folder}test_fold_{fold}{country_suffix}.csv")

    
    # Filter out imagery files that match the source type (L or S)
    available_imagery = [
        os.path.join(imagery_path, d, f)
        for d in os.listdir(imagery_path)
        if d[-2] == imagery_source
        for f in os.listdir(os.path.join(imagery_path, d))
    ]

    def is_available(centroid_id):
        return any(centroid_id in centroid for centroid in available_imagery)

    train_df = train_df[train_df["CENTROID_ID"].apply(is_available)]
    test_df = test_df[test_df["CENTROID_ID"].apply(is_available)]
    if test_df.empty:
        raise Exception("Empty test set")

    # Find the imagery file based on the CENTROID_ID
    def filter_contains(query):
        for item in available_imagery:
            if query in item:
                return item

    train_df["imagery_path"] = train_df["CENTROID_ID"].apply(filter_contains)
    train_df = train_df[train_df["deprived_sev"].notna()]
    test_df["imagery_path"] = test_df["CENTROID_ID"].apply(filter_contains)
    test_df = test_df[test_df["deprived_sev"].notna()]

    # Reset indices after all filtering is complete
    train_df = train_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)

    # NOW extract location features from the final, clean dataframes
    if use_location_features:
        print("Preparing training location features...")
        train_location_features = prepare_location_features(train_df)
        
        print("Preparing test location features...")
        test_location_features = prepare_location_features(test_df)
    else:
        train_location_features = None
        test_location_features = None
        print("Not using location features")

    # Load image files and preprocess (stack bands, normalize, clip)
    def load_and_preprocess_image(path):
        with rasterio.open(path) as src:
            r = src.read(grouped_bands[0])
            g = src.read(grouped_bands[1])
            b = src.read(grouped_bands[2])
            img = np.dstack((r, g, b))
            img = img / normalization * 255.0
        img = np.nan_to_num(img, nan=0, posinf=255, neginf=0)
        return np.clip(img, 0, 255).astype(np.uint8)

    # Data are all prepared, now it is time for testing the model

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load pre-trained model from Facebook DINO hub
    base_model = (
        torch.hub.load("facebookresearch/dino:main", model_name)
        if "dino_" in model_name
        else torch.hub.load("facebookresearch/dinov2", model_name)
    )

    # Wrapper class to add regression head on top of DINO model
    class ViTForRegression(nn.Module):
        def __init__(self, base_model):
            super().__init__()
            self.base_model = base_model
            self.regression_head = nn.Linear(model_output_dim, target_size)

        def forward(self, pixel_values):
            outputs = self.base_model(pixel_values)
            return torch.sigmoid(self.regression_head(outputs))

        def forward_encoder(self, pixel_values):
            return self.base_model(pixel_values)

    model = ViTForRegression(base_model)

    if use_checkpoint:
        state_dict = torch.load(checkpoint)
        model.load_state_dict(state_dict["model_state_dict"])

    # Dataset class for training and testing
    class CustomDataset(Dataset):
        def __init__(self, dataframe, transform):
            self.dataframe = dataframe
            self.transform = transform

        def __len__(self):
            return len(self.dataframe)

        def __getitem__(self, idx):
            item = self.dataframe.iloc[idx]
            image = load_and_preprocess_image(item["imagery_path"])
            image_tensor = self.transform(Image.fromarray(image))
            return image_tensor, item[eval_target]

    # Transform image to proper shape and type for model
    transform = transforms.Compose(
        [transforms.Resize((transform_dim, transform_dim)), transforms.ToTensor()]
    )

    # Dataloader wraps dataset to allow batch loading
    train_dataset = CustomDataset(train_df, transform)
    val_dataset = CustomDataset(test_df, transform)

    # I believe there is a certain format for the class CustomDataset in order to run
    # the function DataLoader
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=4)

    model.to(device)
    model.eval()

    # Extract features from base model for training data
    X_train_visual, y_train = [], []

    for images, targets in tqdm(train_loader):
        images, targets = images.to(device), targets.to(device)
        with torch.no_grad():
            outputs = model.base_model(images)
        X_train_visual.append(outputs.cpu()[0].numpy())
        y_train.append(targets.cpu()[0].numpy())

    # Extract features from base model for test data
    X_test_visual, y_test = [], []
    for images, targets in tqdm(val_loader):
        images, targets = images.to(device), targets.to(device)
        with torch.no_grad():
            outputs = model.base_model(images)
        X_test_visual.append(outputs.cpu()[0].numpy())
        y_test.append(targets.cpu()[0].numpy())

    X_train_visual, y_train = np.array(X_train_visual), np.array(y_train)
    X_test_visual, y_test = np.array(X_test_visual), np.array(y_test)

    # Combine visual and location features
    if use_location_features:
        # Since DataLoader processes samples in order and we reset indices,
        # location features align directly with visual features
        X_train = np.concatenate([X_train_visual, train_location_features], axis=1)
        X_test = np.concatenate([X_test_visual, test_location_features], axis=1)
        
        print(f"\nCombined feature dimensions:")
        print(f"  Visual features: {X_train_visual.shape[1]}")
        print(f"  Location features: {train_location_features.shape[1]}")
        print(f"  Total features: {X_train.shape[1]}")
    else:
        X_train = X_train_visual
        X_test = X_test_visual
        print(f"\nUsing visual features only: {X_train.shape[1]} dimensions")
    

    # Save extracted features and targets to CSV
    results_folder = (
        f"modelling/dino/results/split_{mode}{imagery_source}_{fold}_{grouped_bands}"
        f"{'_loc' if use_location_features else ''}{country_suffix}"
    )
    if not os.path.exists(results_folder):
        os.makedirs(results_folder)
    pd.DataFrame(X_train).to_csv(results_folder + "X_train.csv", index=False)
    pd.DataFrame(y_train, columns=["target"]).to_csv(results_folder + "y_train.csv", index=False)
    pd.DataFrame(X_test).to_csv(results_folder + "X_test.csv", index=False)
    pd.DataFrame(y_test, columns=["target"]).to_csv(results_folder + "y_test.csv", index=False)

    # Save visual features separately for comparison
    pd.DataFrame(X_train_visual).to_csv(results_folder + "X_train_visual_only.csv", index=False)
    pd.DataFrame(X_test_visual).to_csv(results_folder + "X_test_visual_only.csv", index=False)

    # Save location features if used
    if use_location_features:
        pd.DataFrame(train_location_features).to_csv(results_folder + "X_train_location.csv", index=False)
        pd.DataFrame(test_location_features).to_csv(results_folder + "X_test_location.csv", index=False)

    # Ridge Regression with cross-validation to evaluate features
    alphas = np.logspace(-6, 6, 20)

    # Define the pipeline once
    ridge_pipeline = Pipeline([
        ("ridge", RidgeCV(alphas=alphas, cv=5, scoring="neg_mean_absolute_error"))
    ])

    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # COMPARATIVE ANALYSIS
    if use_location_features:
        print("\n=== COMPARATIVE ANALYSIS ===")
        
        # Visual features only
        visual_cv_scores = cross_val_score(
            ridge_pipeline, X_train_visual, y_train, cv=kf, scoring="neg_mean_absolute_error"
        )
        ridge_pipeline.fit(X_train_visual, y_train)
        visual_only_score = np.mean(np.abs(ridge_pipeline.predict(X_test_visual) - y_test))
        
        print(f"Visual features only:")
        print(f"  CV MAE: {-visual_cv_scores.mean():.4f} ± {visual_cv_scores.std():.4f}")
        print(f"  Test MAE: {visual_only_score:.4f}")
        
        # Location features only (if meaningful)
        if train_location_features.shape[1] > 0:
            location_cv_scores = cross_val_score(
                ridge_pipeline, train_location_features, y_train, cv=kf, scoring="neg_mean_absolute_error"
            )
            ridge_pipeline.fit(train_location_features, y_train)
            location_only_score = np.mean(np.abs(ridge_pipeline.predict(test_location_features) - y_test))
            
            print(f"Location features only:")
            print(f"  CV MAE: {-location_cv_scores.mean():.4f} ± {location_cv_scores.std():.4f}")
            print(f"  Test MAE: {location_only_score:.4f}")
        else:
            location_only_score = None
        
        # Combined features
        combined_cv_scores = cross_val_score(
            ridge_pipeline, X_train, y_train, cv=kf, scoring="neg_mean_absolute_error"
        )
        ridge_pipeline.fit(X_train, y_train)
        combined_score = np.mean(np.abs(ridge_pipeline.predict(X_test) - y_test))
        
        print(f"Combined features:")
        print(f"  CV MAE: {-combined_cv_scores.mean():.4f} ± {combined_cv_scores.std():.4f}")
        print(f"  Test MAE: {combined_score:.4f}")
        
        # Calculate improvement
        improvement = visual_only_score - combined_score
        improvement_pct = (improvement / visual_only_score) * 100
        print(f"\nImprovement from adding location: {improvement:.4f} MAE ({improvement_pct:.1f}%)")
        
        # Save detailed analysis
        analysis_results = {
            'visual_only_cv_mae': -visual_cv_scores.mean(),
            'visual_only_cv_std': visual_cv_scores.std(),
            'visual_only_test_mae': visual_only_score,
            'location_only_test_mae': location_only_score,
            'combined_cv_mae': -combined_cv_scores.mean(),
            'combined_cv_std': combined_cv_scores.std(),
            'combined_test_mae': combined_score,
            'improvement_mae': improvement,
            'improvement_pct': improvement_pct,
            'visual_features_count': X_train_visual.shape[1],
            'location_features_count': train_location_features.shape[1],
            'total_features_count': X_train.shape[1]
        }
        
        pd.DataFrame([analysis_results]).to_csv(results_folder + "feature_analysis.csv", index=False)
        
        # Use combined results for final reporting
        final_cv_scores = combined_cv_scores
        final_test_score = combined_score
    
    else:
        # Standard evaluation without location features
        final_cv_scores = cross_val_score(
            ridge_pipeline, X_train, y_train, cv=kf, scoring="neg_mean_absolute_error"
        )
        ridge_pipeline.fit(X_train, y_train)
        final_test_score = np.mean(np.abs(ridge_pipeline.predict(X_test) - y_test))

    # Final results
    print(f"\n=== FINAL EVALUATION RESULTS ===")
    print("Cross-validation scores (negative MAE):", final_cv_scores)
    print("Mean cross-validation score (negative MAE):", final_cv_scores.mean())
    print("Test Score (MAE):", final_test_score)

    return final_test_score


